In [2]:
from openai import OpenAI
from dotenv import load_dotenv
import json
load_dotenv()

True

In [3]:
client = OpenAI()

In [127]:
# These are the functions to be called by the Assistant. These will be API calls to a weather app, 
# however, for simplicity, here I just hardcoded the return values.

def get_current_temperature(location, unit):
    if unit == 'Fahrenheit':
        return '90'
    else:
        return '25'

def get_rain_probability(location):
    return '0.2'

In [97]:

assistant = client.beta.assistants.create(
  name="Weather Forecaster"
  instructions="You are a weather bot. Use the provided functions to answer questions.",
  model="gpt-4o",
  tools=[
    {
      "type": "function",
      "function": {
        "name": "get_current_temperature",
        "description": "Get the current temperature for a specific location",
        "parameters": {
          "type": "object",
          "properties": {
            "location": {
              "type": "string",
              "description": "The city and state, e.g., San Francisco, CA"
            },
            "unit": {
              "type": "string",
              "enum": ["Celsius", "Fahrenheit"],
              "description": "The temperature unit to use. Infer this from the user's location."
            }
          },
          "required": ["location", "unit"]
        }
      }
    },
    {
      "type": "function",
      "function": {
        "name": "get_rain_probability",
        "description": "Get the probability of rain for a specific location",
        "parameters": {
          "type": "object",
          "properties": {
            "location": {
              "type": "string",
              "description": "The city and state, e.g., San Francisco, CA"
            }
          },
          "required": ["location"]
        }
      }
    }
  ]
)

In [100]:
# If we want to discard the previously used thread:
client.beta.threads.delete(thread.id)

ThreadDeleted(id='thread_VWR7hsYnLMqUSULsWcPHa4RP', deleted=True, object='thread.deleted')

In [101]:
# Create a thread
thread = client.beta.threads.create()

In [102]:
thread.id

'thread_8cQ9fTUGXP2bW0uz5oTOukOZ'

In [113]:
# Create our first message. Note that we ask for 2 things, hence 2 function calls are needed.
message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content="What's the weather in Los Angeles today and the likelihood it'll rain?",
)

In [114]:
# Create a run without streaming
# Note that runs expire after 10 minutes.

run = client.beta.threads.runs.create_and_poll(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

In [115]:
run.status

'requires_action'

In [116]:
tool_call_1 = run.required_action.submit_tool_outputs.tool_calls[0]
arguments_1 = json.loads(tool_call_1.function.arguments)
function_name_1 = tool_call_1.function.name
print('Function 1: ', function_name_1)
print('Arguments 1: ', arguments_1, '\n')

tool_call_2 = run.required_action.submit_tool_outputs.tool_calls[1]
arguments_2 = json.loads(tool_call_2.function.arguments)
function_name_2 = tool_call_2.function.name
print('Function 2: ', function_name_2)
print('Arguments 2: ', arguments_2)

Function 1:  get_current_temperature
Arguments 1:  {'location': 'Los Angeles, CA', 'unit': 'Fahrenheit'} 

Function 2:  get_rain_probability
Arguments 2:  {'location': 'Los Angeles, CA'}


In [117]:
# Run the funcitons locally:
temp_result = get_current_temperature(arguments_1['location'], arguments_1['unit'])
print('Temperature: ', temp_result)

rain_result = get_rain_probability(arguments_2['location'])
print('Rain prob: ', rain_result)

Temperature:  90
Rain prob:  0.2


In [118]:
# Define the list to store tool outputs
tool_outputs = []
 
# Loop through each tool in the required action section and append them to tool_outputs.
for tool in run.required_action.submit_tool_outputs.tool_calls:
  if tool.function.name == "get_current_temperature":
    tool_outputs.append({
      "tool_call_id": tool.id,
      "output": temp_result
    })
  elif tool.function.name == "get_rain_probability":
    tool_outputs.append({
      "tool_call_id": tool.id,
      "output": rain_result
    })

In [119]:
tool_outputs

[{'tool_call_id': 'call_JQOzrZeB18IAf3MULtcoMENG', 'output': '90'},
 {'tool_call_id': 'call_lvqAHDhG8nz3mwyXDXlw1H2z', 'output': '0.2'}]

In [120]:
# Submit all tool outputs at once after collecting them in a list
if tool_outputs:
  try:
    run = client.beta.threads.runs.submit_tool_outputs_and_poll(
      thread_id=thread.id,
      run_id=run.id,
      tool_outputs=tool_outputs
    )
    print("Tool outputs submitted successfully.")
  except Exception as e:
    print("Failed to submit tool outputs:", e)
else:
  print("No tool outputs to submit.")
 
if run.status == 'completed':
  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  print(messages)
else:
  print(run.status)

Tool outputs submitted successfully.
SyncCursorPage[Message](data=[Message(id='msg_cjJvN3UbjJrXIj3jQw3pMpBW', assistant_id='asst_fXhgbmuawcZYWSZ41yBJ6td3', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value="In Los Angeles, CA, the current temperature is 90°F, and there's a 20% probability of rain today."), type='text')], created_at=1718535521, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_YJn55hGJZ8L0dOM4BUr3WeQ4', status=None, thread_id='thread_8cQ9fTUGXP2bW0uz5oTOukOZ'), Message(id='msg_Iv1AUPnvnm1nuPu2qIRlG6sR', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value="What's the weather in Los Angeles today and the likelihood it'll rain?"), type='text')], created_at=1718535505, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_8cQ9

In [121]:
messages = client.beta.threads.messages.list(
    thread_id = thread.id,
    order = 'asc'
)

In [122]:
for message in messages:
    print(message.content[0].text.value)
    print('\n')

What's the weather in Windsor, UK today and the likelihood it'll rain?


In Windsor, UK, the current temperature is 25°C, and there's a 20% probability of rain today.


What's the weather in Los Angeles today and the likelihood it'll rain?


In Los Angeles, CA, the current temperature is 90°F, and there's a 20% probability of rain today.




In [39]:
# Delete the assistant:
client.beta.assistants.delete(assistant.id)

AssistantDeleted(id='asst_XlJ5fgKjzjNsaNNtBUcjzxrt', deleted=True, object='assistant.deleted')